# Projeto de C209-L1 ->  JPEG

Alunos: 
Ana Luiza Silva Terra
Gabriela Almada Paranhos de Melo
Rafaela Monteiro Terra




### Historia

* Data de criação 1990;
* Criado por Eric Hamilton (C-Cube Microsystems), e actualmente a Joint Photographic Experts Group.
* Ultima versão lançada dia 27 de Março de 1998;
* Variantes: .jtif ; .jp2 ; .jpx ; .jng;

### Introdução
* JPG (ou JPEG) é um método comum usado para comprimir imagens fotográficas;
* O grau de redução pode ser ajustado, o que permite a você escolher o tamanho de armazenamento e seu compromisso com a     qualidade da imagem. Geralmente se obtém uma compressão pouco perceptível na perda de qualidade da imagem;
* Criado por Eric Hamilton (C-Cube Microsystems), e actualmente a Joint Photographic Experts Group.
* Ultima versão lançada dia 27 de Março de 1998;
* Variantes: .jtif ; .jp2 ; .jpx ; .jng;

### Compressão

Por que comprimir dados?

* Reduz necessidade de atualizar hardware (tal como discos rígidos com mais espaço);
* Melhora eficiência de bancos de dados;
* Backups mais velozes;
* Pode aumentar a segurança de armazenamento (dados relevantes são mais difíceis de serem encontrados).


#### Compressão sem perdas

* Métodos de compressão sem perdas são aqueles que permitem a restauração da informação original em sua totalidade;
* Nenhuma informação é perdida;
* São usados em situações em que perdas não são aceitáveis (e.g. imagens médicas, biometria, aplicações legais etc);
* As vezes são utilizados conjuntamente com processos de compressão com perdas, como o JPEG.

#### Compressão com perda

* Comumente utilizado para compressão de imagens, áudio e vídeo;
* Frequentemente trabalha-se em outro domínio (e.g.: domínio da frequência);
* Tentam se adequar baseado nos limites da percepção;
* Normalmente as perdas são irreversíveis.




### Visão geral do algoritmo

O fluxo de compressão de um JPEG é realizado com os seguintes passos:
* A imagem é convertida para um espaço de cores baseado em crominância, tal como o [YCbCr](https://en.wikipedia.org/wiki/YCbCr);
* Fazer a subamostragem dos componentes de crominância;
    * Estes espaços de cores se baseiam no fato que a visão humana é menos sensível à crominância;
    * Com isso, é possível remover dados referentes à crominância em blocos de tamanhos pré definidos, visto que essa remoção é imperceptível;
* Aplicar a DCT em blocos 8x8;
* Quantizar cada bloco;
* Codificar os dados com algum método de compressão para remover redundâncias, com codificação zigzag, tal como RLE seguido de Huffman.


### Código:



In [5]:
import cv2
import numpy as np
import scipy.fftpack as fftpack
import zlib

def encode_ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def decode_ycbcr(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

def encode_dct(orig, bx, by):
    new_shape = (
        orig.shape[0] // bx * bx,
        orig.shape[1] // by * by,
        3
    )
    new = orig[
        :new_shape[0],
        :new_shape[1]
    ].reshape((
        new_shape[0] // bx,
        bx,
        new_shape[1] // by,
        by,
        3
    ))
    return fftpack.dctn(new, axes=[1,3], norm='ortho')


def decode_dct(orig, bx, by):
    return fftpack.idctn(orig, axes=[1,3], norm='ortho'
    ).reshape((
        orig.shape[0]*bx,
        orig.shape[2]*by,
        3
    ))

def encode_quant(orig, quant):
    return (orig / quant).astype(np.int)


def decode_quant(orig, quant):
    return (orig * quant).astype(float)

def encode_zip(x):
    return zlib.compress(x.astype(np.int8).tobytes())


def decode_zip(orig, shape):
    return np.frombuffer(zlib.decompress(orig), dtype=np.int8).astype(float).reshape(shape)

if __name__ == '__main__':
    im = cv2.imread("inatel.jpeg")
    quants = [5, 10]
    blocks = [(16, 16), (32, 32)]
    for qscale in quants:
        for bx, by in blocks:

            quant = (
                (np.ones((bx, by)) * (qscale * qscale))
                .clip(-100, 100)
                .reshape((1, bx, 1, by, 1))
            )
            ency = encode_ycbcr(im)
            encd = encode_dct(ency, bx, by)
            encq = encode_quant(encd, quant)
            encz = encode_zip(encq)
            decz = decode_zip(encz, encq.shape)
            decq = decode_quant(encq, quant)
            decd = decode_dct(decq, bx, by)
            decy = decode_ycbcr(decd)
            cv2.imwrite("inatel_quant_{}_block_{}x{}.jpeg".format(qscale, bx, by), decy.astype(np.uint8))

### Conclusão:
* O JPEG é uma excelente extensão para se trabalhar e umas das mais populares, muito indicado para fotografia, é o formato predominante das camêras digitais.
* O JPEG quando processado têm uma excelente eficiência em relação ao tamanho da imagem, resultando num arquivo menor.
* Também e compativel com a maioria dos dispositivos e softwares.
* Com isso, o JPEG é um ótimo formato para uma foto.


### Referências:
* https://pt.wikipedia.org/wiki/JPEG
* http://www2.dcc.ufmg.br/disciplinas/ii/ii05-1/seminario/jpeg.pdf
* https://www.alice.cnptia.embrapa.br/alice/bitstream/doc/17224/1/1390.pdf
* https://www.youtube.com/watch?v=E5paqwlM0uw
* https://www.youtube.com/watch?v=PHrYZUN-pW0
* https://github.com/changhsinlee/software-for-science